# TOC -> nbextensions

In [ ]:
import numpy as np
import scipy.fft
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import clear_output, display, HTML
from ipywidgets import interactive as interactive
import cv2

%matplotlib inline

## Visual processing as a cascade of filtering operations

### The receptive field of a visual neuron as a filter

In the first layers of visual processing (retina, LGN, V1, ...), cells are spatially arranged
in a way that adjacent cells respond to partially overlapping portions of the visual field.

We tend to characterize these cells based on their so-called receptive field properties.
The receptive field of a cell is the region of the visual field that makes a cell change its
firing in the presence of a certain visual stimulation.
The type of visual stimulation that a cell preferrably responds to can differ both in their
spatial as well as temporal properties.
Analogously, we can model the behavior of visual cells using spatial and temporal filters.

Let's have a look into some characteristics of cells along the first layers of visual processing.

**TO-DO: Add an image for visualization

### Receptive fields of retinal ganglion cells and cells in LGN

### Receptive fields of cells in V1

## Spatiotemporal properties of the visual system

### The contrast sensitivity function

### The CSF and multi-scale spatial filtering

## Other applications

### Multi-scale spatial filtering and brightness perception